In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#label encoding
from sklearn.preprocessing import LabelEncoder


In [6]:
df=pd.read_csv('archive\IMDB Dataset.csv')

In [7]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [8]:
df.isna().sum()
df.drop_duplicates(inplace=True)

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    #remove html tags
    text = re.sub(r'<.*?>', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    #stopword removal
    # text= ' '.join([word for word in text.split() if word not in stopwords.words("english")])
    text = text.strip()
    return text


In [10]:
df['review'] = df['review'].apply(clean_text)

In [11]:
def stopwordss(text):
    text = ' '.join([word for word in text.split() if word not in stopwords.words("english")])
    return text

In [12]:
#regex method to remove stopwords
from collections import Counter
stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)
def re_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords_dict])
# text = ' '.join([word for word in text.split() if word not in stopwords_dict])

In [30]:
remove_stopwords(' '.join(stopwords.words("english")))

"   myself  our ours ourselves you you' you' you' you' your yours yourself yourselves  him his himself she she' her hers herself  ' its itself they them their theirs themselves what which who whom this that that' these those   are was were  been being have has had having  does did doing   the and but   because  until while    for with about against between into through during before after above below  from  down  out  off over under again further then once here there when where why how all any both each few more most other some such  nor not only own same  than too very   can will just don don' should should' now        ain aren aren' couldn couldn' didn didn' doesn doesn' hadn hadn' hasn hasn' haven haven' isn isn'  mightn mightn' mustn mustn' needn needn' shan shan' shouldn shouldn' wasn wasn' weren weren' won won' wouldn wouldn'"

In [13]:
df['review'] = df['review'].apply(re_stopwords)

In [14]:
lb=LabelEncoder()
df['sentiment']=lb.fit_transform(df['sentiment'])

In [15]:
max_length = max(df['review'].apply(lambda x: len(x.split())))

In [16]:
(max_length )*0.6

852.0

In [17]:
tk=keras.preprocessing.text.Tokenizer(num_words=10000)
tk.fit_on_texts(df['review'])
X=tk.texts_to_sequences(df['review'])
X=keras.preprocessing.sequence.pad_sequences(X,maxlen=600)


In [20]:
def pre_process_data(data):
    data = data.apply(clean_text)
    data = data.apply(re_stopwords)
    return data

In [22]:
def pad_data(data):
    
    X = keras.preprocessing.sequence.pad_sequences(data, maxlen=600)
    return X

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)

In [71]:
#model
model = keras.Sequential()
model.add(layers.Embedding(10000, 128, input_length=600))
model.add(layers.Bidirectional(layers.GRU(128)))
# model.add(layers.GRU(128))
model.add(layers.Dropout(0.4))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='leaky_relu'))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 600, 128)          1280000   
                                                                 
 bidirectional_8 (Bidirectio  (None, 256)              198144    
 nal)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_17 (Dense)            (None, 64)                16448     
                                                                 
 dense_18 (Dense)            (None, 2)               

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=[early_stopping])


In [25]:
from tensorflow.keras.models import load_model
model=load_model('best_model_bi.h5')